In [37]:
import pandas as pd
import json
from glob import glob

1. train : 학습용 데이터셋

	├ 10001 : 데이터 고유 아이디

	│	├ 10001.jpg : 이미지 파일

	│	├ 10001.csv : 환경 데이터

	│		└ 촬영 전 48 시간의 "측정 시각", "내부 온도", "내부 습도", "내부 이슬점", "내부 CO2", "외부 풍속", "외부 누적일사" 등의 환경 정보

	│	└ 10001.json :

	│		├ description 

	│		│	├ image : 이미지 파일 이름

	│		│	├ date : 촬영 날짜

	│		│	├ time : 촬영 시간

	│		│	├ region : 촬영 지역

	│		│	├ height : 이미지 높이

	│		│	├ width : 이미지 너비

	│		│	└  task : 데이터 종류 (질병/해충/병해/정상 구분)

	│		└ annotations

	│				├ disease : 작물 상태 코드

	│				├ crop : 작물 코드

	│				├ area : 작물 촬영 부위

	│				├ grow : 작물의 생육 단계 

	│				├ risk : 질병 피해 정도

	│				├ bbox : 주목 객체 바운딩 박스 (x, y, w, h 형태)

	│				└ part : 병해 부위 바운딩 박스 (x, y, w, h 형태)

	│

	├ 10002

	├ 10003

	└ ...

	[추가] train.csv : train set에 대한 정답 파일

 image : 이미지 파일 이름
 label : "{작물 코드}_{작물 상태 코드}_{질병 피해 정도}" 형태의 문자열
 "{crop}_{disease}_{risk}"




2. test : 평가용 데이터셋

	├ 10001 : 데이터 고유 아이디

	│	├ 10001.csv : 환경 데이터

	│		└ 촬영 전 48 시간의 "측정 시각", "내부 온도", "내부 습도", "내부 이슬점", "내부 CO2", "외부 풍속", "외부 누적일사" 등의 환경 정보

	│	└ 10001.jpg : 이미지 파일

	│

	├ 10002

	├ 10003

	└ ...



3. sample_submission.csv : 제출용 양식

 image : 이미지 파일 이름
 **label : "{작물 코드}_{작물 상태 코드}_{질병 피해 정도}" 형태의 문자열**
 "{crop}_{disease}_{risk}"

 ### purpose 
 "작물 환경 데이터"와 "작물 병해 이미지"를 이용해 "작물의 종류", "병해의 종류", "병해의 진행 정도"를 진단하는 AI 모델 개발



In [36]:
#part(질병이 있는 데이터에만 표시 없는 데이터는 [])
L = 39506
json_path = f'작물데이터/train/{L}/{L}.json'
with open(json_path,'r') as f:
    json_data = json.load(f)
json_data

{'description': {'date': '20210817',
  'image': '39506.jpg',
  'task': 2,
  'width': 384,
  'time': '1108',
  'region': 0,
  'height': 512},
 'annotations': {'area': 1,
  'disease': 'b5',
  'grow': 24,
  'bbox': [{'x': 30.96837410094246,
    'h': 447.83324032738096,
    'y': 43.527715773809526,
    'w': 273.72237723214283}],
  'part': [{'x': 196.1143043154762,
    'h': 31.200106181795636,
    'y': 49.71446010044643,
    'id': 0,
    'w': 20.571490575396826},
   {'x': 62.05669100322421,
    'h': 18.514353918650794,
    'y': 114.8575458829365,
    'id': 1,
    'w': 18.51435004340278},
   {'x': 200.914310515873,
    'h': 11.65718005952381,
    'y': 99.7717788938492,
    'id': 2,
    'w': 17.142919146825395},
   {'x': 79.54246496775794,
    'h': 11.314329117063492,
    'y': 227.31507316468253,
    'id': 3,
    'w': 9.600035652281745},
   {'x': 103.19969153025794,
    'h': 8.914326016865079,
    'y': 233.14366319444446,
    'id': 4,
    'w': 8.571459573412698}],
  'risk': 1,
  'crop': 6}}

In [30]:
#측정시간 간격은 10min but row count는 모두 다름. 
csv_path = f'작물데이터/train/{L}/{L}.csv'
csv_data = pd.read_csv(csv_path)
csv_data

,측정시각,내부 온도 1 평균,내부 온도 1 최고,내부 온도 1 최저,내부 온도 2 평균,내부 온도 2 최고,내부 온도 2 최저,내부 온도 3 평균,내부 온도 3 최고,내부 온도 3 최저,...,배지 중량 최저,양액 온도 평균,양액 온도 최고,양액 온도 최저,외부 풍향 수치,외부 풍향,외부 빗물 시간,외부 누적일사 평균,양액 급액 누적,양액 배액 누적
0,2021-10-18 16:10:00,21.8,22.0,21.5,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
1,2021-10-18 16:00:00,22.3,22.4,22.0,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
2,2021-10-18 15:50:00,21.6,22.1,21.0,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
3,2021-10-18 15:40:00,20.9,21.0,20.7,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
4,2021-10-18 15:30:00,21.7,22.4,20.9,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,2021-10-16 16:00:00,22.0,22.0,21.9,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
73,2021-10-16 15:50:00,21.9,21.9,21.9,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
74,2021-10-16 15:40:00,21.9,22.0,21.9,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
75,2021-10-16 15:30:00,22.0,22.0,22.0,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-


In [23]:
# -가 null 판정을 받지않기 때문에 describe에 나타나는 col로 판단.
csv_data.describe()

,내부 온도 1 평균,내부 온도 1 최고,내부 온도 1 최저,내부 습도 1 평균,내부 습도 1 최고,내부 습도 1 최저,내부 이슬점 평균,내부 이슬점 최고,내부 이슬점 최저,내부 CO2 평균,내부 CO2 최고,내부 CO2 최저
count,245.000000,245.000000,245.000000,245.000000,245.000000,245.000000,245.000000,245.000000,245.000000,245.000000,245.000000,245.000000
mean,21.295918,21.381633,21.202857,82.679184,83.418776,81.964898,18.094694,18.296735,17.880816,441.823673,445.926531,437.853061
std,3.235330,3.272355,3.199198,11.558952,11.018229,12.027397,1.364424,1.482103,1.289173,39.139803,39.370283,38.792571
min,16.600000,16.600000,16.500000,59.700000,60.700000,58.400000,15.200000,15.200000,15.000000,382.300000,386.000000,380.000000
25%,18.800000,18.800000,18.700000,70.900000,73.900000,69.000000,17.900000,17.900000,17.800000,399.400000,403.000000,396.000000
50%,20.100000,20.200000,20.100000,89.400000,89.900000,88.800000,18.200000,18.400000,18.000000,454.100000,459.000000,448.000000
75%,24.700000,24.800000,24.600000,92.600000,92.800000,92.500000,19.000000,19.200000,18.700000,477.200000,481.000000,473.000000
max,27.200000,27.300000,26.900000,94.100000,94.200000,94.000000,21.000000,21.500000,20.400000,507.500000,513.000000,506.000000


In [39]:
sample = glob('작물데이터/train/*')[42]
sample

'작물데이터/train\\10519'